# Component 4: FAISS Library Benchmark

## About FAISS

**FAISS (Facebook AI Similarity Search)** is a library developed by Meta AI Research for efficient similarity search and clustering of dense vectors. It's designed for billion-scale datasets and is widely used in production systems.

### Key Features:

1. **Multiple Index Types:**
   - **Flat (Exact)**: Brute-force exact search (baseline)
   - **IVF (Inverted File)**: Clustering-based (similar to our VQ)
   - **IVFPQ**: Combines IVF with Product Quantization
   - **HNSW**: Hierarchical Navigable Small World graphs
   - **LSH**: Locality Sensitive Hashing

2. **Optimizations:**
   - GPU acceleration support
   - SIMD vectorization for CPU
   - Compressed storage (PQ, SQ)
   - Multi-threaded search

3. **Suitable For:**
   - High-dimensional dense vectors (embeddings)
   - Large-scale datasets (millions to billions)
   - Real-time similarity search
   - Image/text/audio retrieval systems

### Comparison to Our Implementations:

- **IndexIVFFlat**: Similar to our VQ (K-means clustering)
- **IndexIVFPQ**: Combines clustering + product quantization (like VQ + PQ)
- **IndexLSH**: Similar to our LSH implementation
- **IndexFlatL2**: Exact search baseline

FAISS is production-ready with extensive optimizations, making it 10-100x faster than naive implementations.

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import faiss

np.random.seed(42)
np.set_printoptions(precision=4, suppress=True)

In [ ]:
doc_vectors = np.load("Data/processed/doc_vectors_w2v.npy").astype('float32')
metadata = pd.read_csv("Data/processed/doc_metadata.csv")

print(f"Vectors: {doc_vectors.shape}")
print(f"Metadata: {metadata.shape[0]} records")
print(f"FAISS version: {faiss.__version__}")

## Helper Functions

In [ ]:
def compute_ndcg_at_k(predicted, actual, k):
    """Normalized Discounted Cumulative Gain."""
    dcg = sum([1/np.log2(i+2) for i, doc in enumerate(predicted[:k]) if doc in actual[:k]])
    idcg = sum([1/np.log2(i+2) for i in range(min(len(actual), k))])
    return dcg / idcg if idcg > 0 else 0

def compute_recall_at_k(predicted, actual, k):
    """Recall@k."""
    return len(set(predicted[:k]) & set(actual[:k])) / k

def compute_ground_truth(vectors, query_indices, k=10):
    """Compute exact nearest neighbors using FAISS Flat index."""
    d = vectors.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(vectors)
    
    ground_truth = {}
    for qi in query_indices:
        _, I = index.search(vectors[qi:qi+1], k)
        ground_truth[qi] = I[0]
    
    return ground_truth

## Precompute Ground Truth

In [ ]:
TOP_K = 10
N_QUERIES = 50

query_indices = np.random.choice(len(doc_vectors), N_QUERIES, replace=False)

print("Computing ground truth with exact search...")
exact_results = compute_ground_truth(doc_vectors, query_indices, TOP_K)
print("✓ Ground truth computed")

## Experiment 1: Accuracy vs Efficiency

We test three FAISS index types with varying parameters:
- **IndexIVFFlat**: Inverted file index (like VQ)
- **IndexIVFPQ**: IVF with Product Quantization
- **IndexLSH**: Locality Sensitive Hashing

In [ ]:
results = []
d = doc_vectors.shape[1]
n_clusters = 100

print("\n=== Testing IndexIVFFlat (IVF) ===")

# Train quantizer
quantizer = faiss.IndexFlatL2(d)
index_ivf = faiss.IndexIVFFlat(quantizer, d, n_clusters)
index_ivf.train(doc_vectors)
index_ivf.add(doc_vectors)

for nprobe in [1, 2, 5, 10, 20]:
    print(f"  Testing nprobe={nprobe}")
    index_ivf.nprobe = nprobe
    
    recalls = []
    ndcgs = []
    query_times = []
    
    for qi in query_indices:
        t0 = time.perf_counter()
        _, I = index_ivf.search(doc_vectors[qi:qi+1], TOP_K)
        query_times.append(time.perf_counter() - t0)
        
        recalls.append(compute_recall_at_k(I[0], exact_results[qi], TOP_K))
        ndcgs.append(compute_ndcg_at_k(I[0], exact_results[qi], TOP_K))
    
    results.append({
        "method": "FAISS-IVF",
        "index_type": "IndexIVFFlat",
        "n_clusters": n_clusters,
        "nprobe": nprobe,
        "recall_at_k": np.mean(recalls),
        "ndcg_at_k": np.mean(ndcgs),
        "candidate_ratio": nprobe / n_clusters,
        "query_time": np.mean(query_times),
        "N": len(doc_vectors),
        "dim": d,
    })

df_exp1 = pd.DataFrame(results)
print("\n=== IVF Results ===")
display(df_exp1)

In [ ]:
print("\n=== Testing IndexIVFPQ (IVF + PQ) ===")

m = 8  # number of subquantizers
nbits = 8  # bits per subquantizer

quantizer = faiss.IndexFlatL2(d)
index_ivfpq = faiss.IndexIVFPQ(quantizer, d, n_clusters, m, nbits)
index_ivfpq.train(doc_vectors)
index_ivfpq.add(doc_vectors)

for nprobe in [1, 2, 5, 10, 20]:
    print(f"  Testing nprobe={nprobe}")
    index_ivfpq.nprobe = nprobe
    
    recalls = []
    ndcgs = []
    query_times = []
    
    for qi in query_indices:
        t0 = time.perf_counter()
        _, I = index_ivfpq.search(doc_vectors[qi:qi+1], TOP_K)
        query_times.append(time.perf_counter() - t0)
        
        recalls.append(compute_recall_at_k(I[0], exact_results[qi], TOP_K))
        ndcgs.append(compute_ndcg_at_k(I[0], exact_results[qi], TOP_K))
    
    results.append({
        "method": "FAISS-IVFPQ",
        "index_type": "IndexIVFPQ",
        "n_clusters": n_clusters,
        "nprobe": nprobe,
        "recall_at_k": np.mean(recalls),
        "ndcg_at_k": np.mean(ndcgs),
        "candidate_ratio": nprobe / n_clusters,
        "query_time": np.mean(query_times),
        "N": len(doc_vectors),
        "dim": d,
    })

df_exp1 = pd.DataFrame(results)
print("\n=== IVFPQ Results ===")
display(df_exp1[df_exp1["method"] == "FAISS-IVFPQ"])

In [ ]:
print("\n=== Testing IndexLSH ===")

for nbits in [64, 128, 256, 512]:
    print(f"  Testing nbits={nbits}")
    index_lsh = faiss.IndexLSH(d, nbits)
    index_lsh.add(doc_vectors)
    
    recalls = []
    ndcgs = []
    query_times = []
    
    for qi in query_indices:
        t0 = time.perf_counter()
        _, I = index_lsh.search(doc_vectors[qi:qi+1], TOP_K)
        query_times.append(time.perf_counter() - t0)
        
        recalls.append(compute_recall_at_k(I[0], exact_results[qi], TOP_K))
        ndcgs.append(compute_ndcg_at_k(I[0], exact_results[qi], TOP_K))
    
    results.append({
        "method": "FAISS-LSH",
        "index_type": "IndexLSH",
        "n_clusters": None,
        "nprobe": nbits,
        "recall_at_k": np.mean(recalls),
        "ndcg_at_k": np.mean(ndcgs),
        "candidate_ratio": 1.0,
        "query_time": np.mean(query_times),
        "N": len(doc_vectors),
        "dim": d,
    })

df_exp1 = pd.DataFrame(results)
print("\n=== LSH Results ===")
display(df_exp1[df_exp1["method"] == "FAISS-LSH"])

In [ ]:
print("\n=== Experiment 1: All FAISS Methods ===")
display(df_exp1)

## Experiment 2: Scaling with N

In [ ]:
N_LIST = [1000, 2000, 5000, 10000, min(20000, len(doc_vectors))]
TEST_QUERIES = 10
BEST_NPROBE = 5

scaling_results = []

for N in N_LIST:
    print(f"\nTesting N={N}")
    X = doc_vectors[:N]
    test_idx = np.random.choice(N, TEST_QUERIES, replace=False)
    
    # IVFFlat
    quantizer = faiss.IndexFlatL2(d)
    index = faiss.IndexIVFFlat(quantizer, d, min(100, N//10))
    
    t0 = time.perf_counter()
    index.train(X)
    index.add(X)
    build_t = time.perf_counter() - t0
    
    index.nprobe = BEST_NPROBE
    q_times = []
    for qi in test_idx:
        t1 = time.perf_counter()
        index.search(X[qi:qi+1], TOP_K)
        q_times.append(time.perf_counter() - t1)
    
    scaling_results.append({
        "method": "FAISS-IVF",
        "N": N,
        "dim": d,
        "build_time": build_t,
        "query_time": np.mean(q_times),
    })
    
    # IVFPQ
    quantizer = faiss.IndexFlatL2(d)
    index_pq = faiss.IndexIVFPQ(quantizer, d, min(100, N//10), 8, 8)
    
    t0 = time.perf_counter()
    index_pq.train(X)
    index_pq.add(X)
    build_t = time.perf_counter() - t0
    
    index_pq.nprobe = BEST_NPROBE
    q_times = []
    for qi in test_idx:
        t1 = time.perf_counter()
        index_pq.search(X[qi:qi+1], TOP_K)
        q_times.append(time.perf_counter() - t1)
    
    scaling_results.append({
        "method": "FAISS-IVFPQ",
        "N": N,
        "dim": d,
        "build_time": build_t,
        "query_time": np.mean(q_times),
    })
    
    # LSH
    index_lsh = faiss.IndexLSH(d, 128)
    
    t0 = time.perf_counter()
    index_lsh.add(X)
    build_t = time.perf_counter() - t0
    
    q_times = []
    for qi in test_idx:
        t1 = time.perf_counter()
        index_lsh.search(X[qi:qi+1], TOP_K)
        q_times.append(time.perf_counter() - t1)
    
    scaling_results.append({
        "method": "FAISS-LSH",
        "N": N,
        "dim": d,
        "build_time": build_t,
        "query_time": np.mean(q_times),
    })

df_exp2 = pd.DataFrame(scaling_results)
print("\n=== Experiment 2: Scaling with N ===")
display(df_exp2)

## Experiment 3: Scaling with Dimensionality

In [ ]:
DIM_LIST = [50, 100, 200]
N_SAMPLE = 10000
TEST_QUERIES = 10

dim_results = []
X_sample = doc_vectors[:N_SAMPLE]

for dim in DIM_LIST:
    print(f"\nTesting d={dim}")
    X = X_sample[:, :dim]
    test_idx = np.random.choice(N_SAMPLE, TEST_QUERIES, replace=False)
    
    # IVFFlat
    quantizer = faiss.IndexFlatL2(dim)
    index = faiss.IndexIVFFlat(quantizer, dim, 100)
    
    t0 = time.perf_counter()
    index.train(X)
    index.add(X)
    build_t = time.perf_counter() - t0
    
    index.nprobe = BEST_NPROBE
    q_times = []
    for qi in test_idx:
        t1 = time.perf_counter()
        index.search(X[qi:qi+1], TOP_K)
        q_times.append(time.perf_counter() - t1)
    
    dim_results.append({
        "method": "FAISS-IVF",
        "N": N_SAMPLE,
        "dim": dim,
        "build_time": build_t,
        "query_time": np.mean(q_times),
    })
    
    # IVFPQ
    m = max(2, dim // 25)
    quantizer = faiss.IndexFlatL2(dim)
    index_pq = faiss.IndexIVFPQ(quantizer, dim, 100, m, 8)
    
    t0 = time.perf_counter()
    index_pq.train(X)
    index_pq.add(X)
    build_t = time.perf_counter() - t0
    
    index_pq.nprobe = BEST_NPROBE
    q_times = []
    for qi in test_idx:
        t1 = time.perf_counter()
        index_pq.search(X[qi:qi+1], TOP_K)
        q_times.append(time.perf_counter() - t1)
    
    dim_results.append({
        "method": "FAISS-IVFPQ",
        "N": N_SAMPLE,
        "dim": dim,
        "build_time": build_t,
        "query_time": np.mean(q_times),
    })
    
    # LSH
    nbits = dim * 2
    index_lsh = faiss.IndexLSH(dim, nbits)
    
    t0 = time.perf_counter()
    index_lsh.add(X)
    build_t = time.perf_counter() - t0
    
    q_times = []
    for qi in test_idx:
        t1 = time.perf_counter()
        index_lsh.search(X[qi:qi+1], TOP_K)
        q_times.append(time.perf_counter() - t1)
    
    dim_results.append({
        "method": "FAISS-LSH",
        "N": N_SAMPLE,
        "dim": dim,
        "build_time": build_t,
        "query_time": np.mean(q_times),
    })

df_exp3 = pd.DataFrame(dim_results)
print("\n=== Experiment 3: Scaling with Dimensionality ===")
display(df_exp3)

## Save Results

In [ ]:
results_dir = "Data/results"
os.makedirs(results_dir, exist_ok=True)

df_exp1.to_csv(f"{results_dir}/faiss_accuracy_efficiency.csv", index=False)
df_exp2.to_csv(f"{results_dir}/faiss_scaling_N.csv", index=False)
df_exp3.to_csv(f"{results_dir}/faiss_scaling_dim.csv", index=False)

print("✓ All FAISS results saved")

## Visualization

In [ ]:
import seaborn as sns

sns.set(style="whitegrid")

# Plot Recall vs Candidate Ratio for Experiment 1
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_exp1, x="candidate_ratio", y="recall_at_k", hue="method", marker="o")
plt.title("FAISS: Recall@K vs Candidate Ratio")
plt.xlabel("Candidate Ratio")
plt.ylabel("Recall@K")
plt.xscale('log')
plt.ylim(0, 1)
plt.legend(title="Method")
plt.show()